<a href="https://colab.research.google.com/github/apchavezr/prueba_trabajo/blob/main/Modelo_Regression_Docker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contenerización de un modelo de regresión lineal con Docker

Este cuaderno muestra cómo encapsular un modelo de *machine learning* (regresión lineal) usando Docker, permitiendo su ejecución consistente en desarrollo, pruebas y producción.

## 1. Estructura del Proyecto

Cree una carpeta llamada `modelo_regresion_docker` y dentro de ella los siguientes archivos:

- `train_model.py` — entrena y guarda el modelo
- `app.py` — expone una API para usar el modelo
- `requirements.txt` — dependencias del entorno
- `Dockerfile` — instrucciones para crear la imagen Docker

## 2. Código del modelo (train_model.py)

In [1]:
# train_model.py
import pandas as pd
from sklearn.linear_model import LinearRegression
import joblib

# Crear dataset simulado
X = pd.DataFrame({'horas_estudio': [1, 2, 3, 4, 5]})
y = pd.Series([2, 4, 6, 8, 10])  # relación lineal simple

# Entrenar modelo
modelo = LinearRegression()
modelo.fit(X, y)

# Guardar modelo
joblib.dump(modelo, 'modelo.pkl')
print("Modelo entrenado y guardado.")

Modelo entrenado y guardado.


## 3. API con Flask (app.py)

In [ ]:
# app.py
from flask import Flask, request, jsonify
import joblib
import pandas as pd

app = Flask(__name__)
modelo = joblib.load('modelo.pkl')

@app.route('/predecir', methods=['POST'])
def predecir():
    datos = request.get_json()
    df = pd.DataFrame(datos)
    predicciones = modelo.predict(df)
    return jsonify(predicciones.tolist())

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

## 4. Dependencias del entorno (requirements.txt)

In [ ]:
flask
scikit-learn
pandas
joblib

## 5. Dockerfile

In [ ]:
# Dockerfile
FROM python:3.10

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

RUN python train_model.py

EXPOSE 5000

CMD ["python", "app.py"]

## 6. Construcción y ejecución del contenedor Docker

### a. Construir la imagen
```bash
docker build -t modelo_regresion .
```

### b. Ejecutar el contenedor
```bash
docker run -p 5000:5000 modelo_regresion
```

## 7. Probar la API

Con `curl` o Postman:

```bash
docker run -p 5000:5000 modelo_regresion
curl -X POST http://localhost:5000/predecir -H "Content-Type: application/json" -d "[{\"horas_estudio\": 6}]"
```

Resultado esperado:
```json
[12.0]
```

## ✅ Conclusión

Docker permite empaquetar aplicaciones analíticas asegurando que se ejecuten consistentemente en cualquier entorno, lo cual es útil para el despliegue confiable de modelos de *machine learning* en producción.